# Nonparametric ML Models - Cumulative Lab

## Introduction

In this cumulative lab, you will apply two nonparametric models you have just learned — k-nearest neighbors and decision trees — to the forest cover dataset.

## Objectives

* Practice identifying and applying appropriate preprocessing steps
* Perform an iterative modeling process, starting from a baseline model
* Explore multiple model algorithms, and tune their hyperparameters
* Practice choosing a final model across multiple model algorithms and evaluating its performance

## Your Task: Complete an End-to-End ML Process with Nonparametric Models on the Forest Cover Dataset

![line of pine trees](https://curriculum-content.s3.amazonaws.com/data-science/images/trees.jpg)

Photo by <a href="https://unsplash.com/@michaelbenz?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Michael Benz</a> on <a href="/s/photos/forest?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>

### Business and Data Understanding

To repeat the previous description:

> Here we will be using an adapted version of the forest cover dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/covertype). Each record represents a 30 x 30 meter cell of land within Roosevelt National Forest in northern Colorado, which has been labeled as `Cover_Type` 1 for "Cottonwood/Willow" and `Cover_Type` 0 for "Ponderosa Pine". (The original dataset contained 7 cover types but we have simplified it.)

The task is to predict the `Cover_Type` based on the available cartographic variables:

In [1]:
# Run this cell without changes
import pandas as pd

df = pd.read_csv('data/forest_cover.csv')
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2553,235,17,351,95,780,188,253,199,1410,...,0,0,0,0,0,0,0,0,0,0
1,2011,344,17,313,29,404,183,211,164,300,...,0,0,0,0,0,0,0,0,0,0
2,2022,24,13,391,42,509,212,212,134,421,...,0,0,0,0,0,0,0,0,0,0
3,2038,50,17,408,71,474,226,200,102,283,...,0,0,0,0,0,0,0,0,0,0
4,2018,341,27,351,34,390,152,188,168,190,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,0
38497,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,0
38498,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,0
38499,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,0


> As you can see, we have over 38,000 rows, each with 52 feature columns and 1 target column:

> * `Elevation`: Elevation in meters
> * `Aspect`: Aspect in degrees azimuth
> * `Slope`: Slope in degrees
> * `Horizontal_Distance_To_Hydrology`: Horizontal dist to nearest surface water features in meters
> * `Vertical_Distance_To_Hydrology`: Vertical dist to nearest surface water features in meters
> * `Horizontal_Distance_To_Roadways`: Horizontal dist to nearest roadway in meters
> * `Hillshade_9am`: Hillshade index at 9am, summer solstice
> * `Hillshade_Noon`: Hillshade index at noon, summer solstice
> * `Hillshade_3pm`: Hillshade index at 3pm, summer solstice
> * `Horizontal_Distance_To_Fire_Points`: Horizontal dist to nearest wildfire ignition points, meters
> * `Wilderness_Area_x`: Wilderness area designation (3 columns)
> * `Soil_Type_x`: Soil Type designation (39 columns)
> * `Cover_Type`: 1 for cottonwood/willow, 0 for ponderosa pine

This is also an imbalanced dataset, since cottonwood/willow trees are relatively rare in this forest:

In [3]:
# Run this cell without changes
print("Raw Counts")
print(df["Cover_Type"].value_counts())
print()
print("Percentages")
print(df["Cover_Type"].value_counts(normalize=True))

Raw Counts
Cover_Type
0    35754
1     2747
Name: count, dtype: int64

Percentages
Cover_Type
0    0.928651
1    0.071349
Name: proportion, dtype: float64


Thus, a baseline model that always chose the majority class would have an accuracy of over 92%. Therefore we will want to report additional metrics at the end.

### Previous Best Model

In a previous lab, we used SMOTE to create additional synthetic data, then tuned the hyperparameters of a logistic regression model to get the following final model metrics:

* **Log loss:** 0.13031294393913376
* **Accuracy:** 0.9456679825472678
* **Precision:** 0.6659919028340081
* **Recall:** 0.47889374090247455

In this lab, you will try to beat those scores using more-complex, nonparametric models.

### Modeling

Although you may be aware of some additional model algorithms available from scikit-learn, for this lab you will be focusing on two of them: k-nearest neighbors and decision trees. Here are some reminders about these models:

#### kNN - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

This algorithm — unlike linear models or tree-based models — does not emphasize learning the relationship between the features and the target. Instead, for a given test record, it finds the most similar records in the training set and returns an average of their target values.

* **Training speed:** Fast. In theory it's just saving the training data for later, although the scikit-learn implementation has some additional logic "under the hood" to make prediction faster.
* **Prediction speed:** Very slow. The model has to look at every record in the training set to find the k closest to the new record.
* **Requires scaling:** Yes. The algorithm to find the nearest records is distance-based, so it matters that distances are all on the same scale.
* **Key hyperparameters:** `n_neighbors` (how many nearest neighbors to find; too few neighbors leads to overfitting, too many leads to underfitting), `p` and `metric` (what kind of distance to use in defining "nearest" neighbors)

#### Decision Trees - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

Similar to linear models (and unlike kNN), this algorithm emphasizes learning the relationship between the features and the target. However, unlike a linear model that tries to find linear relationships between each of the features and the target, decision trees look for ways to split the data based on features to decrease the entropy of the target in each split.

* **Training speed:** Slow. The model is considering splits based on as many as all of the available features, and it can split on the same feature multiple times. This requires exponential computational time that increases based on the number of columns as well as the number of rows.
* **Prediction speed:** Medium fast. Producing a prediction with a decision tree means applying several conditional statements, which is slower than something like logistic regression but faster than kNN.
* **Requires scaling:** No. This model is not distance-based. You also can use a `LabelEncoder` rather than `OneHotEncoder` for categorical data, since this algorithm doesn't necessarily assume that the distance between `1` and `2` is the same as the distance between `2` and `3`.
* **Key hyperparameters:** Many features relating to "pruning" the tree. By default they are set so the tree can overfit, and by setting them higher or lower (depending on the hyperparameter) you can reduce overfitting, but too much will lead to underfitting. These are: `max_depth`, `min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, `max_features`, `max_leaf_nodes`, and `min_impurity_decrease`. You can also try changing the `criterion` to "entropy" or the `splitter` to "random" if you want to change the splitting logic.

### Requirements

#### 1. Prepare the Data for Modeling

#### 2. Build a Baseline kNN Model

#### 3. Build Iterative Models to Find the Best kNN Model

#### 4. Build a Baseline Decision Tree Model

#### 5. Build Iterative Models to Find the Best Decision Tree Model

#### 6. Choose and Evaluate an Overall Best Model

## 1. Prepare the Data for Modeling

The target is `Cover_Type`. In the cell below, split `df` into `X` and `y`, then perform a train-test split with `random_state=42` and `stratify=y` to create variables with the standard `X_train`, `X_test`, `y_train`, `y_test` names.

Include the relevant imports as you go.

In [19]:
# Your code here
#import library
from sklearn.model_selection import train_test_split
#split data sets
X = df.drop('Cover_Type', axis=1)
y = df['Cover_Type']
#split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Now, instantiate a `StandardScaler`, fit it on `X_train`, and create new variables `X_train_scaled` and `X_test_scaled` containing values transformed with the scaler.

In [21]:
# Your code here
#import Library
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#fit scaler to trained data 
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

The following code checks that everything is set up correctly:

In [23]:
# Run this cell without changes

# Checking that df was separated into correct X and y
assert type(X) == pd.DataFrame and X.shape == (38501, 52)
assert type(y) == pd.Series and y.shape == (38501,)

# Checking the train-test split
assert type(X_train) == pd.DataFrame and X_train.shape == (28875, 52)
assert type(X_test) == pd.DataFrame and X_test.shape == (9626, 52)
assert type(y_train) == pd.Series and y_train.shape == (28875,)
assert type(y_test) == pd.Series and y_test.shape == (9626,)

# Checking the scaling
assert X_train_scaled.shape == X_train.shape
assert round(X_train_scaled[0][0], 3) == -0.636
assert X_test_scaled.shape == X_test.shape
assert round(X_test_scaled[0][0], 3) == -1.370

AssertionError: 

## 2. Build a Baseline kNN Model

Build a scikit-learn kNN model with default hyperparameters. Then use `cross_val_score` with `scoring="neg_log_loss"` to find the mean log loss for this model (passing in `X_train_scaled` and `y_train` to `cross_val_score`). You'll need to find the mean of the cross-validated scores, and negate the value (either put a `-` at the beginning or multiply by `-1`) so that your answer is a log loss rather than a negative log loss.

Call the resulting score `knn_baseline_log_loss`.

Your code might take a minute or more to run.

In [25]:
# Replace None with appropriate code
# Imports
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Creating the model
knn_baseline_model = KNeighborsClassifier()

# Perform cross-validation
knn_baseline_log_loss = cross_val_score(knn_baseline_model, X_train_scaled, y_train, 
                                        cv=5, scoring="neg_log_loss")

knn_baseline_log_loss

array([-0.12556677, -0.15768765, -0.1073781 , -0.07888703, -0.11880624])

Our best logistic regression model had a log loss of 0.13031294393913376

Is this model better? Compare it in terms of metrics and speed.

# Replace None with appropriate text
### Comparing kNN vs Logistic Regression

- **Log Loss**:  
  - **kNN** has a log loss of **0.1256**, which is slightly better than **Logistic Regression**'s **0.1303**, indicating better accuracy.
  
- **Speed**:  
  - **kNN** is slower at predicting because it computes distances to all training points, while **Logistic Regression** is faster due to its simpler linear model.

### Conclusion:
- **kNN** performs slightly better in terms of **log loss**, but **Logistic Regression** is faster at making predictions.


## 3. Build Iterative Models to Find the Best kNN Model

Build and evaluate at least two more kNN models to find the best one. Explain why you are changing the hyperparameters you are changing as you go. These models will be *slow* to run, so be thinking about what you might try next as you run them.

In [29]:
# Your code here (add more cells as needed)
#Import libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# MODEL 1
knn_model_1 = KNeighborsClassifier(n_neighbors=3)
knn_model_1_scores = cross_val_score(knn_model_1, X_train_scaled, y_train, cv=5, scoring="neg_log_loss")

# Calculate mean log loss for model 1
knn_model_1_log_loss = -1 * np.mean(knn_model_1_scores)
print(f"Model 1 (n_neighbors=3) Log Loss: {knn_model_1_log_loss}")


Model 1 (n_neighbors=3) Log Loss: 0.19896566347222702


In [30]:
# Your code here (add more cells as needed)
# Model2
knn_model_2 = KNeighborsClassifier(n_neighbors=7)
knn_model_2_scores = cross_val_score(knn_model_2, X_train_scaled, y_train, cv=5, scoring="neg_log_loss")

# Calculate mean log loss for model 2
knn_model_2_log_loss = -1 * np.mean(knn_model_2_scores)
print(f"Model 2 (n_neighbors=7) Log Loss: {knn_model_2_log_loss}")

Model 2 (n_neighbors=7) Log Loss: 0.09382049081801118


In [33]:
# Your code here (add more cells as needed)
# Model 3
knn_model_3 = KNeighborsClassifier(n_neighbors=5, metric='manhattan')
knn_model_3_scores = cross_val_score(knn_model_3, X_train_scaled, y_train, cv=5, scoring="neg_log_loss")

# Calculate mean log loss for model 3
knn_model_3_log_loss = -1 * np.mean(knn_model_3_scores)
print(f"Model 3 (n_neighbors=5, metric='manhattan') Log Loss: {knn_model_3_log_loss}")


Model 3 (n_neighbors=5, metric='manhattan') Log Loss: 0.11684356290711453


In [37]:
# model4
knn_model_4 = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_model_4_scores = cross_val_score(knn_model_4, X_train_scaled, y_train, cv=5, scoring="neg_log_loss")

# Calculate mean log loss for model 4
knn_model_4_log_loss = -1 * np.mean(knn_model_4_scores)
print(f"Model 4 (n_neighbors=5, metric='euclidean') Log Loss: {knn_model_4_log_loss}")


Model 4 (n_neighbors=5, metric='euclidean') Log Loss: 0.1176651591100023


## 4. Build a Baseline Decision Tree Model

Now that you have chosen your best kNN model, start investigating decision tree models. First, build and evaluate a baseline decision tree model, using default hyperparameters (with the exception of `random_state=42` for reproducibility).

(Use cross-validated log loss, just like with the previous models.)

In [39]:
# Your code here
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Decision Tree
decision_tree_model = DecisionTreeClassifier(random_state=42)

#  cross-validation 
decision_tree_scores = cross_val_score(decision_tree_model, X_train_scaled, y_train, cv=5, scoring="neg_log_loss")

# Calculate the mean log loss 
decision_tree_log_loss = -1 * np.mean(decision_tree_scores)

# results
decision_tree_log_loss


0.7161920738357047

Interpret this score. How does this compare to the log loss from our best logistic regression and best kNN models? Any guesses about why?

# Replace None with appropriate text
The **log loss of 0.7162** for the baseline Decision Tree model is significantly higher than the log loss values from both the **best logistic regression model (0.1303)** and the **best kNN model (0.1256)**. This indicates that the Decision Tree model is performing worse in terms of **prediction accuracy**.

## 5. Build Iterative Models to Find the Best Decision Tree Model

Build and evaluate at least two more decision tree models to find the best one. Explain why you are changing the hyperparameters you are changing as you go.

In [52]:
# Your code here (add more cells as needed)
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Model1
decision_tree_model_1 = DecisionTreeClassifier(random_state=42, max_depth=5)
decision_tree_model_1_scores = cross_val_score(decision_tree_model_1, X_train_scaled, y_train, cv=5, scoring="neg_log_loss")

# Calculate mean log loss for model 1
decision_tree_model_1_log_loss = -1 * np.mean(decision_tree_model_1_scores)
decision_tree_model_1_log_loss


0.11136362559813533

In [54]:
# Your code here (add more cells as needed)
# Model2
decision_tree_model_2 = DecisionTreeClassifier(random_state=42, min_samples_split=10)
decision_tree_model_2_scores = cross_val_score(decision_tree_model_2, X_train_scaled, y_train, cv=5, scoring="neg_log_loss")

# Calculate mean log loss for model 2
decision_tree_model_2_log_loss = -1 * np.mean(decision_tree_model_2_scores)
decision_tree_model_2_log_loss 

0.497061760167787

In [56]:
# Your code here (add more cells as needed)
#model3
decision_tree_model_3 = DecisionTreeClassifier(random_state=42, min_samples_leaf=10, max_depth=7)
decision_tree_model_3_scores = cross_val_score(decision_tree_model_3, X_train_scaled, y_train, cv=5, scoring="neg_log_loss")

# Calculate mean log loss for model 3
decision_tree_model_3_log_loss = -1 * np.mean(decision_tree_model_3_scores)
decision_tree_model_3_log_loss

0.11361860947845812

## 6. Choose and Evaluate an Overall Best Model

Which model had the best performance? What type of model was it?

Instantiate a variable `final_model` using your best model with the best hyperparameters.

In [58]:
# Replace None with appropriate code
final_model = KNeighborsClassifier(n_neighbors=7)

# Fit the model on the full training data
# (scaled or unscaled depending on the model)
final_model.fit(X_train_scaled, y_train)



KNeighborsClassifier(n_neighbors=7)

Now, evaluate the log loss, accuracy, precision, and recall. This code is mostly filled in for you, but you need to replace `None` with either `X_test` or `X_test_scaled` depending on the model you chose.

In [62]:
# Replace None with appropriate code
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss

preds = final_model.predict(X_test_scaled)
probs = final_model.predict_proba(X_test_scaled)

print("log loss: ", log_loss(y_test, probs))
print("accuracy: ", accuracy_score(y_test, preds))
print("precision:", precision_score(y_test, preds))
print("recall:   ", recall_score(y_test, preds))


log loss:  0.10700890021862704
accuracy:  0.9823399558498896
precision: 0.9041095890410958
recall:    0.8415300546448088


Interpret your model performance. How would it perform on different kinds of tasks? How much better is it than a "dummy" model that always chooses the majority class, or the logistic regression described at the start of the lab?

# Replace None with appropriate text
The kNN model performs exceptionally well with a log loss of 0.107, accuracy of 98.2%, precision of 90.4%, and recall of 84.2%. This indicates it's highly effective for binary classification tasks, especially where distinguishing between classes is crucial.


## Conclusion

In this lab, you practiced the end-to-end machine learning process with multiple model algorithms, including tuning the hyperparameters for those different algorithms. You saw how nonparametric models can be more flexible than linear models, potentially leading to overfitting but also potentially reducing underfitting by being able to learn non-linear relationships between variables. You also likely saw how there can be a tradeoff between speed and performance, with good metrics correlating with slow speeds.